<h1 div class='alert alert-success'><center> Ponto de partida (EDA)</center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05)

# Descrição de dados

Para esta competição, você vai prever se um cliente fez uma reclamação sobre uma apólice de seguro. A verdade fundamental claimtem valor binário, mas uma previsão pode ser qualquer número de 0.0 para 1.0, representando a probabilidade de uma reclamação. Os recursos neste conjunto de dados foram tornados anônimos e podem conter valores ausentes.
arquivos

- `train.csv`: os dados de treinamento com o alvo claimcoluna
- `test.csv`: o conjunto de teste; você estará prevendo o claimpara cada linha neste arquivo
- `sample_submission.csv`:  um arquivo de envio de amostra no formato correto

# <div class="alert alert-success">  1. IMPORTAÇÕES </div> 

## 1.1. Bibliotecas 

In [ ]:
import warnings
import random
import os
import gc
import torch

In [ ]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt 
import seaborn           as sns
import joblib            as jb
import xgboost           as xgb

In [ ]:
from sklearn.model_selection import train_test_split,  KFold, StratifiedKFold
from sklearn.preprocessing   import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.preprocessing   import QuantileTransformer, LabelEncoder
from sklearn.impute          import SimpleImputer
from sklearn                 import metrics
from datetime                import datetime

## 1.2. Funções
Aqui centralizamos todas as funções desenvolvidas durante o projeto para melhor organização do código.

In [ ]:
def jupyter_setting():
    
    %matplotlib inline
      
    #os.environ["WANDB_SILENT"] = "true" 
    #plt.style.use('bmh') 
    #plt.rcParams['figure.figsize'] = [20,15]
    #plt.rcParams['font.size']      = 13
     
    pd.options.display.max_columns = None
    #pd.set_option('display.expand_frame_repr', False)

    warnings.filterwarnings(action='ignore')
    warnings.simplefilter('ignore')
    warnings.filterwarnings('ignore')
    #warnings.filterwarnings(category=UserWarning)

    warnings.filterwarnings('ignore', category=DeprecationWarning)
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    #warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

    pd.set_option('display.max_rows', 150)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.max_colwidth', None)

    icecream = ["#00008b", "#960018","#008b00", "#00468b", "#8b4500", "#582c00"]
    #sns.palplot(sns.color_palette(icecream))
    
    return icecream

icecream = jupyter_setting()

# Colors
dark_red = "#b20710"
black    = "#221f1f"
green    = "#009473"
myred    = '#CD5C5C'
myblue   = '#6495ED'
mygreen  = '#90EE90'

cols= [myred, myblue,mygreen]

In [ ]:
colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen",
          "lightseagreen", "cornflowerblue", "mediumpurple", "palevioletred",
          "lightskyblue", "sandybrown", "yellowgreen", "indianred",
          "lightsteelblue", "mediumorchid", "deepskyblue"]

In [ ]:
def missing_zero_values_table(df):
        mis_val         = df.isnull().sum()
        mis_val_percent = round(df.isnull().mean().mul(100), 2)
        mz_table        = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table        = mz_table.rename(columns = {df.index.name:'col_name', 
                                                     0 : 'Valores ausentes', 
                                                     1 : '% de valores totais'})
        
        mz_table['Tipo de dados'] = df.dtypes
        mz_table                  = mz_table[mz_table.iloc[:,1] != 0 ]. \
                                     sort_values('% de valores totais', ascending=False)
        
        msg = "Seu dataframe selecionado tem {} colunas e {} " + \
              "linhas. \nExistem {} colunas com valores ausentes."
            
        print (msg.format(df.shape[1], df.shape[0], mz_table.shape[0]))
        
        return mz_table.reset_index()

In [ ]:
def describe(df):
    var = df.columns

    # Medidas de tendência central, média e mediana 
    ct1 = pd.DataFrame(df[var].apply(np.mean)).T
    ct2 = pd.DataFrame(df[var].apply(np.median)).T

    # Dispensão - str, min , max range skew, kurtosis
    d1 = pd.DataFrame(df[var].apply(np.std)).T
    d2 = pd.DataFrame(df[var].apply(min)).T
    d3 = pd.DataFrame(df[var].apply(max)).T
    d4 = pd.DataFrame(df[var].apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(df[var].apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(df[var].apply(lambda x: x.kurtosis())).T
    d7 = pd.DataFrame(df[var].apply(lambda x: (3 *( np.mean(x) - np.median(x)) / np.std(x) ))).T

    # concatenete 
    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6, d7]).T.reset_index()
    m.columns = ['attrobutes', 'min', 'max', 'range', 'mean', 'median', 'std','skew', 'kurtosis','coef_as']
    
    return m

In [ ]:
def reduce_memory_usage(df, verbose=True):
    
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    
    for col in df.columns:
        
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
        
    return df

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, "r-", label=label)
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.rcParams['font.size'] = 12
    plt.title('ROC curve for TPS 09')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.legend(loc="lower right")
    plt.grid(True)

In [ ]:
def confusion_plot(matrix, labels = None, title = None):
        
    labels = labels if labels else ['Negative (0)', 'Positive (1)']    
    
    fig, ax = plt.subplots(nrows=1, ncols=1)
    
    sns.heatmap(data        = matrix, 
                cmap        = 'Blues', 
                annot       = True, 
                fmt         = 'd',
                xticklabels = labels, 
                yticklabels = labels, 
                ax          = ax);
    
    ax.set_xlabel('\n PREVISTO', fontsize=15)
    ax.set_ylabel('REAL \n', fontsize=15)
    ax.set_title(title)
    
    plt.close();
    
    return fig;

In [ ]:
def df_corr(df, annot_=False):
    
    df = df.corr(method ='pearson').round(5)

    # Máscara para ocultar a parte superior direita do gráfico, pois é uma duplicata
    mask = np.zeros_like(df)
    mask[np.triu_indices_from(mask)] = True

    # Making a plot
    plt.figure(figsize=(15,12))
    ax = sns.heatmap(df, annot=annot_, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})

    ax.set_title("Mapa de calor de correlação das variável", fontsize=17)

    plt.setp(ax.get_xticklabels(), 
             rotation      = 90, 
             ha            = "right",
             rotation_mode = "anchor", 
             weight        = "normal")

    plt.setp(ax.get_yticklabels(), 
             weight        = "normal",
             rotation_mode = "anchor", 
             rotation      = 0, 
             ha            = "right");

In [ ]:
def graf_outlier(df, feature):
    col = [(0,4), (5,9)]

    df_plot = ((df[feature] - df[feature].min())/
               (df[feature].max() - df[feature].min()))

    fig, ax = plt.subplots(len(col), 1, figsize=(15,7))

    for i, (x) in enumerate(col): 
        sns.boxplot(data = df_plot.iloc[:, x[0]:x[1] ], ax = ax[i]); 

In [ ]:
def diff(t_a, t_b):
    from dateutil.relativedelta import relativedelta
    t_diff = relativedelta(t_b, t_a)  # later/end time comes first!
    return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)

In [ ]:
def free_gpu_cache():
    
    # https://www.kaggle.com/getting-started/140636
    #print("Initial GPU Usage")
    #gpu_usage()                             

    #cuda.select_device(0)
    #cuda.close()
    #cuda.select_device(0)   
    
    gc.collect()
    torch.cuda.empty_cache()

## 1.3. Dataset

### 1.3.1. Carregar Dados
São dois arquivos que vamos utilizar para análise e treinanmento dos modelos, e um arquivo para submissão na competição.

- `train.csv`: arquivo com dados de treinamento;  
- `test.csv`: arquivo que será utilizado para previsão; 
- `sample_submission.csv`: arquivo utlizado para envio das previsões.  


In [ ]:
path   = '../input/tabular-playground-series-dec-2021/'
target = 'Cover_Type'

In [ ]:
df1_train     = pd.read_csv(path + 'train.csv')
df1_test      = pd.read_csv(path + 'test.csv')
df_submission = pd.read_csv(path + 'sample_submission.csv')

df1_train.drop('Id', axis=1, inplace=True)
df1_test.drop('Id', axis=1, inplace=True)

df1_train.shape, df1_test.shape, df_submission.shape

In [ ]:
df1_train.head()

In [ ]:
df1_test.head()

### 1.3.1. Redução dos datasets

In [ ]:
df1_train = reduce_memory_usage(df1_train)
df1_test  = reduce_memory_usage(df1_test)

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
Tivemos uma redução de 85% em ambos os datasets.
    
</div>

In [ ]:
#jb.dump(df1_train, path + 'Data/df_train_nb_01.pkl.z')
#jb.dump(df1_test, path + 'Data/df_test_nb_01.pkl.z')

# <div class="alert alert-success"> 2. Análise Exploratória de Dados (EDA)  </div> 

## 2.1. Dimensão do DataSet

In [ ]:
print('TREINO')
print('Number of Rows: {}'.format(df1_train.shape[0]))
print('Number of Columns: {}'.format(df1_train.shape[1]), end='\n\n')

print('TESTE')
print('Number of Rows: {}'.format(df1_test.shape[0]))
print('Number of Columns: {}'.format(df1_test.shape[1]))

## 2.2. Tipo de dados

In [ ]:
df1_train.info()

In [ ]:
df1_test.info()

In [ ]:
print(f'{3*"="} For Pandas {10*"="}\n{(df1_train.dtypes).value_counts()}')
print(f'\n{3*"="} For Datatable {7*"="}\n{(df1_test.dtypes).value_counts()}')

## 2.3. identificar variáveis ausentes (NA)

In [ ]:
missing = missing_zero_values_table(df1_train)
missing[:].style.background_gradient(cmap='Reds')

In [ ]:
missing = missing_zero_values_table(df1_test)
missing[:].style.background_gradient(cmap='Reds')

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>

Não temos dados faltantes.
    
</div>

## 2.4 Estatística Descritiva
Abaixo estão as estatísticas básicas para cada variável que contém informações sobre contagem, média, desvio padrão, mínimo, 1º quartil, mediana, 3º quartil e máximo.

In [ ]:
feature_float = []
feature_cat = df1_test.filter(regex=r'Wilderness_Area').columns.to_list() + \
              df1_test.filter(regex=r'Soil_Type').columns.to_list()

for col in df1_test.columns:
    if col not in feature_cat: 
        feature_float.append(col)   

feature_float_test = feature_float.copy()
feature_float.append('Cover_Type')

print('Temos {} variávies numéricas e {} categóricas.'.format(len(feature_float), 
                                                              len(feature_cat)))

### 2.4.1 Atributos numéricos

- Train

In [ ]:
df1_train[feature_float].describe().T.style.background_gradient(cmap='YlOrRd')

- Test

In [ ]:
df1_test[feature_float_test].describe().T.style.background_gradient(cmap='YlOrRd')

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>

O que me chamou atenção é valor negativo para a maioria das variáveis exceto para `Hillshade_Noon` em ambos os dataset e para a variável `Hillshade_9am` apenas no dataset de test, pode ser um indicativo de algum problema na coleta de informações, mas com se trata de informações de `Cobertura Florestal` pode fazer sentido. É um ponto que temos que levar em consideração no processamento de dados.  
    
</div>

### 2.4.2. Atributos categóricos

In [ ]:
df1_train[feature_cat].columns

In [ ]:
df1_train.filter(regex=r'Wil').head()

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
    
Temos quatros variáveis que podemos transformar em dammy, isso pode ajudar a melhor a performance dos modelos. 

    
</div> 

## 2.5. Correlação
Vamos examinar a correlação entre as variáveis.

In [ ]:
df_corr(df1_train[feature_float], annot_=True)

In [ ]:
df_corr(df1_train)

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
    
Como podemos observar, a correlação está entre -0.6 e 0.2, o que é muito pequeno, portanto, as variáveis são fracamente correlacionados.
    
</div>

## 2.6. Distribuição

### 2.6.1. Train / Test

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

pie = ax.pie([len(df1_train), len(df1_test)],
             labels   = ["Train dataset", "Test dataset"],
             colors   = ["salmon", "teal"],
             textprops= {"fontsize": 15},
             autopct  = '%1.1f%%')

ax.axis("equal")
ax.set_title("Comparação de comprimento do conjunto de dados \n", fontsize=18)
fig.set_facecolor('white')
plt.show();

### 2.6.2. Proporção de variáveis

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

plt.pie([ len(feature_cat), len(feature_float)], 
        labels=['Categorical', 'Continuos' ],
        textprops={'fontsize': 13},
        autopct='%1.1f%%')

#ax.axis("equal")
ax.set_title("Comparação variáveis continuas/categóricas \n Dataset Treino/Teste", fontsize=18)
fig.set_facecolor('white')
plt.show()

### 2.6.3. Variável predidora

In [ ]:

plt.figure(figsize=(8, 4))
ax = sns.countplot(x=df1_train['Cover_Type'], palette='viridis')
ax.set_title('Distribuição da variável Cover_Type', fontsize=20, y=1.05)

sns.despine(right=True)
sns.despine(offset=10, trim=True)

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
Observamos no gráfico acima que a classe 2 é a mais representativa, seguida pela classe 1. As classes raras são 4, 5 e 6, a classe 5 com apenas um único exemplo podemos fazer a exclusão ou associá-la a outra classe, mas para isso devemos analisar a associação dessa classe, pois terá um impacto mínimo na classificação.  
    
</div>

### 2.6.4. Distribuição Train x Test

In [ ]:
col = [(0,5), (5,9)]
       
for x in col:
       
    L    = len(df1_train[feature_float_test].columns[x[0]:x[1]])
    nrow = int(np.ceil(L/6))
    ncol = 5
    i    = 1

    remove_last = (nrow * ncol) - L
    fig, ax     = plt.subplots(nrow, ncol,figsize=(20, 4))
    
    fig.subplots_adjust(top=0.8)
    
    for feature in df1_train[feature_float_test].columns[x[0]:x[1]]:
        plt.subplot(nrow, ncol, i)
        ax = sns.kdeplot(df1_train[feature], shade=True, color='salmon',  alpha=0.5, label='train')
        ax = sns.kdeplot(df1_test[feature], shade=True, color='teal',  alpha=0.5, label='test')
        plt.xlabel(feature, fontsize=10)
        plt.legend()

        i += 1
        
        gc.collect()
    
    plt.suptitle('DistPlot: train & test data de {} à {}'.format(x[0],x[1]), fontsize=15)
    plt.show();

### 2.6.5.  Variáveis categoricas

In [ ]:
fig, axes = plt.subplots(10, 4, figsize=(20, 16))

target_order = sorted(df1_train['Cover_Type'].unique())
mean         = df1_train.groupby('Cover_Type').mean().sort_index()
std          = df1_train.groupby('Cover_Type').std().sort_index()

for idx, ax in zip(range(1,40), axes.flatten()):
    
    ax.bar(mean[f'Soil_Type{idx}'].index, 
           mean[f'Soil_Type{idx}'],           
           width=0.6)
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.margins(0.1)
    
    ax.spines['left'].set_visible(False)
    ax.set_title(f'Soil_Type_{idx}', loc='center', weight='bold', fontsize=11)

axes.flatten()[-1].axis('off')    
axes.flatten()[-2].axis('off')

#fig.supxlabel('Média por classe (por variável)', ha='center', fontweight='bold')

fig.tight_layout()
plt.show()

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
Variáveis `Soil_Type_7` e `Soil_Type_15` não tem informações, neste caso podemos excluir.    
    
</div>

## 2.6.6. Detecção de Outlier

### 2.6.6.1. Data Train 

In [ ]:
graf_outlier(df1_train, feature_float)

### 2.6.6.2. Data Test

In [ ]:
graf_outlier(df1_test, feature_float_test)

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
    
Acima observamos que temos que fazer um tratamento de outliers em algumas variáveis, na etapa de processamento vamos fazer o tramento ou remoção dos outliers para ajudar na previsão dos modelos. 

</div>

# <div class="alert alert-success"> 3. Modelagem (baseline) </div> 

## 3.1. Split Train/Test

In [ ]:
%%time
df1_train[target].iloc[df1_train[df1_train[target]==5].index] = 4
df1_train[target].iloc[df1_train[df1_train[target]==4].index] = 6
df1_train[target].value_counts()

X      = df1_train.drop([target], axis=1)
y      = df1_train[target].astype(int)
X_test = df1_test

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size    = 0.3,
                                                      shuffle      = True,                                                       
                                                      random_state = 12359)

del df1_train,df1_test

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape , X_test.shape

## 3.2. Parametros do modelo

In [ ]:
seed   = 12359
params = {"objective"     :"multi:softmax",    
          'eval_metric'   : 'mlogloss',         
          'random_state'  : seed}

if torch.cuda.is_available():           
    params.update({'tree_method' : 'gpu_hist',                    
                   'predictor'   : 'gpu_predictor'
                   })
params

## 3.2. Seleção de Scaler

In [ ]:
%%time 

model_baseline = xgb.XGBClassifier(**params)

scalers = [StandardScaler(), 
           RobustScaler(), 
           MinMaxScaler(), 
           MaxAbsScaler(), 
           QuantileTransformer(output_distribution='normal', random_state=0)]

for scaler in scalers: 
    
    if scaler!=None:
        X_train_s = scaler.fit_transform(X_train)
        X_valid_s = scaler.fit_transform(X_valid)
    else:
        X_train_s = X_train
        X_valid_s = X_valid
                
    model_baseline.fit(X_train_s, y_train, verbose=False)

    y_pred = model_baseline.predict(X_valid_s)  
    acc    = metrics.accuracy_score(y_valid, y_pred)    
        
    print('Validaçao ACC: {:2.5f} => {}'.format(acc, scaler))

    free_gpu_cache()

print()

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
    
Com scaler  RobustScaler obtivemos uma ACC de 0.92970, como estamos fazer apenas uma validação simples, neste caso a pontuação do score pode ser afetada por aleatoriedade dos dados, sendo assim, vamos fazer uma validação cruzada para termos uma estimativa robusta.  <br>
    
</div>

In [ ]:
def cross_val_model(model_, model_name_, X_, y_, X_test_, target_, scalers_, fold_=5, path_='', seed_=12359):
    
    n_estimators = model_.get_params()['n_estimators']
    score        = []     
    taco         = 50 
    acc_best     = 0

    for scaler_ in scalers_: 

        time_start   = datetime.now()
                
        if scaler_!=None:
            string_scaler = str(scaler_)        
            string_scaler = string_scaler[:string_scaler.index('(')]
            X_tst = scaler_.fit_transform(X_test_.copy())
        else:
            string_scaler = None 
            X_tst = X_test_.copy()
            
        y_pred_test = np.zeros(len(X_test_))

        folds = KFold(n_splits=fold_, random_state=seed_, shuffle=True)

        print('='*taco)
        print('Scaler: {} - n_estimators: {}'.format(string_scaler, n_estimators))
        print('='*taco)

        for fold, (trn_idx, val_idx) in enumerate(folds.split(X_)):

            time_fold_start = datetime.now()

            # ---------------------------------------------------- 
            # Separar dados para treino 
            # ----------------------------------------------------     
            X_trn, X_val = X_.iloc[trn_idx], X_.iloc[val_idx]
            y_trn, y_val = y_.iloc[trn_idx], y_.iloc[val_idx]   
            
            # ---------------------------------------------------- 
            # Processamento 
            # ----------------------------------------------------     
            if scaler_!=None:                
                X_trn = scaler_.fit_transform(X_trn)
                X_val = scaler_.fit_transform(X_val)
                
            # ---------------------------------------------------- 
            # Treinar o modelo 
            # ---------------------------------------------------- 
            model_.fit(X_trn, y_trn,
                    eval_set              = [(X_val, y_val)],                    
                    early_stopping_rounds = n_estimators*.2,
                    verbose               = False)
            
            # ---------------------------------------------------- 
            # Predição 
            # ----------------------------------------------------     
            y_pred_val   = model_.predict(X_val)    
            y_pred_test += model_.predict(X_tst) / folds.n_splits
            
            # ---------------------------------------------------- 
            # Score 
            # ---------------------------------------------------- 
            acc  = metrics.accuracy_score(y_val, y_pred_val)
            score.append(acc)

            # ---------------------------------------------------- 
            # Print resultado  
            # ---------------------------------------------------- 
            time_fold_end = diff(time_fold_start, datetime.now())
            
            print('[Fold {}] ACC: {:2.5f} - {}'.format(fold+1, acc, time_fold_end))

        acc_mean = np.mean(score) 
        acc_std  = np.std(score)

        if acc_mean > acc_best:     
            acc_best      = acc_mean           
            model_best    = model_    
            scaler_best   = scaler_

        time_end = diff(time_start, datetime.now())   

        print('-'*taco)
        print('[Mean Fold] ACC: {:2.5f} std: {:2.5f} - {}'.format(acc_mean, acc_std, time_end))
        print('='*taco)
        print()

        df_submission[target_] = y_pred_test.astype('int')
        name_file_sub = model_name_ + '_' + str(scaler_).lower()[:4] + '.csv'
        df_submission.to_csv(path_ + name_file_sub.format(acc_mean), index = False)

    print('-'*taco)
    print('Scaler Best: {}'.format(scaler_best))
    print('Score      : {:2.5f}'.format(acc_best))
    print('-'*taco)
    print()

In [ ]:
%%time 
scalers = [None,
           StandardScaler(), 
           RobustScaler(), 
           MinMaxScaler(), 
           MaxAbsScaler(), 
           QuantileTransformer(output_distribution='normal', random_state=0)
          ] 

cross_val_model(model_      = xgb.XGBClassifier(**params, learning_rate=0.4),
                model_name_ = 'xgb_baseline_score_{:2.5f}',
                X_          = X,
                y_          = y,
                X_test_     = X_test,
                target_     = target,
                scalers_    = scalers,
                fold_       = 5, 
                path_       = '',
                seed_       = seed
                )

<div class="alert alert-info" role="alert"> 
 
**`CONCLUSÃO:`** <br>
    
1. Verificar os valores negativos identificados na análise descritiva dos dados; 
2. Transformação das variáveis `Wilderness_Area` em dammy;
3. Tratamento de outliers; 
4. Exclusão das variáveis `Soil_Type_7` e `Soil_Type_15` sem informações para modelagem. 
    
    
</div>